In [2]:
import time
import copy

from pathlib import Path

import tqdm

# 1. Request station location

In [3]:
def request_metro_station_location(data, city, key):
    
    from codes.utils.request_data.request_data_amap import geocoding

    all_res = []
    
    for ix, row in tqdm.tqdm(data.iterrows()):
        
        ln = row['line_name_zh']
        sn = row['name_zh']
        
        # search value
        search_value = f'{city}{sn}(地铁站)'
    
        # request result
        time.sleep(0.05)
        req_res = geocoding(search_value, city=city, key=key)
        
        # request matching
        for res in req_res:
            
            # successfully matching
            if (res['level'] == '公交地铁站点') and (f'{sn}(地铁站)' in res['formatted_address']):
                
                final_res = res
                # additional information
                final_res.update({
                    'line_name_zh' : ln,
                    'name_zh'      : sn,
                    'name'         : row['name'],
                    'line_name'    : row['line_name']})
                
                all_res.append(copy.deepcopy(final_res))
                
                break
    
    res = pd.DataFrame(all_res)
    
    return res

#### (1) Read station list

In [3]:
from codes.utils.load_data.load_data_metro import load_metro_station_list


# load metro station list
path = Path.cwd().parent / 'Shanghai/metro_network_shanghai.xlsx'
data = load_metro_station_list(path, time_status='current')


# load metro station list
# path = Path.cwd().parent / 'Suzhou/metro_network_suzhou.xlsx'
# data = load_metro_station_list(path, time_status='current')

Loading data: 22it [00:01, 11.65it/s]


#### (2) Request station location

In [4]:
key = 'd53f9929db781a22f5710539218ce798'

city = '上海市'
res = request_metro_station_location(data, city, key)
print('Shape of data:', data.shape)

res.to_csv('station_location_gcj02_shanghai.csv', index=False)


# city = '江苏省苏州市'
# res = request_metro_station_location(data, city, key)
# print('Shape of data:', data.shape)
# 
# res.to_csv('station_location_gcj02_suzhou.csv', index=False)

519it [08:44,  1.01s/it]

Shape of data: (519, 10)


In [2]:
from codes.utils.request_data.request_data_amap import geocoding

key = 'd53f9929db781a22f5710539218ce798'
city = '苏州市'
sn = '金鸡墩(地铁站)'

search_value = f'{city}{sn}'
print(search_value)

geocoding(search_value, city=city, key=key)

苏州市金鸡墩(地铁站)


[{'search_value': '苏州市金鸡墩(地铁站)',
  'formatted_address': '江苏省苏州市姑苏区金鸡墩(地铁站)',
  'country': '中国',
  'province': '江苏省',
  'city': '苏州市',
  'citycode': '0512',
  'district': '姑苏区',
  'adcode': '320508',
  'location': '120.567781,31.333255',
  'level': '公交地铁站点'}]

# 2. Update station location information

In [7]:
from pathlib import Path

import pandas as pd

In [8]:
city = 'shanghai'
data_folder = Path.cwd().parent / city

# excel file
excel_path = data_folder / f'metro_network_{city}.xlsx'


# location information
path = data_folder / f'station_location_gcj02_{city}.csv'
loc = pd.read_csv(path)[['location', 'line_name', 'name']] \
        .rename(columns={'location':'location_gcj02'})

In [9]:
# load all excel data
data_all = pd.read_excel(excel_path, sheet_name=None)
# line info
line_info = data_all['line_info']

sheet_name_li = line_info['sheet_name'].tolist()

with pd.ExcelWriter(excel_path, mode='a', if_sheet_exists='replace') as writer:
    
    for sheet_name in sheet_name_li:
        
        data = data_all[sheet_name] \
                .assign(sheet_name=sheet_name) \
                .merge(line_info[['sheet_name', 'line_name', 'line_name_zh']], on='sheet_name', how='left', validate='many_to_one')
        
        # merge location information
        data = data.merge(loc, on=['line_name', 'name'], how='left', validate='one_to_one') \
                   .drop(columns=['sheet_name', 'line_name', 'line_name_zh'], axis=1)
        
        # update sheet 
        data.to_excel(writer, index=False, sheet_name=sheet_name)